In [ ]:
import numpy as np
from ss_problem import SSProblem, SecureStateReconst


# define system model and measurement model
Ac = np.array([[0, 1, 0, 0],[0, -0.2, 0, 0],[0,0,0,1],[0,0,0,-0.2]])
Bc = np.array([[0, 0],[1, 0],[0, 0],[0,1]])
Cc = np.array([[1,0,0,0],[1,0,0,0],[0,1,0,0],[0,1,0,0],[0,0,1,0],[0,0,1,0],[0,0,0,1],[0,0,0,1]])
Dc = np.zeros((Cc.shape[0],Bc.shape[1]))
p = np.shape(Cc)[0]
s = 3
ss_problem = SSProblem(Ac,Bc,Cc,Dc,s)

# define attacking model
fake_state_count = 1
init_states = np.array([[1.,2.],[1.,2.],[1.,2.],[1.,2.]])
att_ind = [0,2,4] # sensors 1, 3, 5
# which initial state and its corresponding sensors 
# 0 -- healthy sensor, 1, 2, ... -- attacked sensors
att_dic = {0:[i for i in range(p) if i not in att_ind], 1:att_ind}

ss_problem.gen_attack_measurement(s=3,att_dic = att_dic,fake_state_count=fake_state_count,
                                    init_states=init_states,noise=False)
print(f'A: {ss_problem.A},  \n B: {ss_problem.B}, \n C:{ss_problem.C}, \n D:{ss_problem.D}')
print('input_sequence:',ss_problem.u_seq)
print('measurement_y:',ss_problem.tilde_y_his)
print('(To be determined) true tates:',ss_problem.init_states[:,0])
print('(To be determined) fake tates:',ss_problem.init_states[:,1])
print(f'(To be determined) Attacked sensor(s) {ss_problem.att_dic.get(1)}')

In [2]:
ssr_solution = SecureStateReconst(ss_problem)


AttributeError: 'SecureStateReconst' object has no attribute 'solve'

In [4]:
for item in Ac:
    print(item)

[0. 1. 0. 0.]
[ 0.  -0.2  0.   0. ]
[0. 0. 0. 1.]
[ 0.   0.   0.  -0.2]


In [ ]:
%timeit possible_states,corresp_sensors, _ = ssr_solution.solve(error_bound = 1e-2)


In [ ]:
if possible_states is not None:
    for ind in range(corresp_sensors.shape[0]):
        sensors = corresp_sensors[ind,:]
        state = possible_states[:,ind]
        print(f'Identified possible states:{state} for sensors {sensors}')

In [ ]:
import timeit

execution_time = timeit.timeit(lambda: ssr_solution.solve(error_bound = 1e-2), number=100)

print(execution_time)


0.36949859996093437


In [ ]:
att_dic

{0: [1, 3, 5, 6, 7], 1: [0, 2, 4]}

In [ ]:
print(np.finfo(np.double).precision)

15


In [ ]:
residuals_list = [1,2,3,4]
residual_min = min(residuals_list)
comb_list = residual_min<2*residual_min

In [ ]:
residuals_list[0]

1